In [ ]:
from sklearn import svm
from sklearn.decomposition import PCA
from skimage.transform import resize
from skimage.io import imread
import pathlib

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
import datetime
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from PIL import Image
from sklearn.decomposition import PCA
from skimage.transform import resize
from skimage.io import imread
from sklearn import metrics

In [ ]:
IMAGE_WIDTH = 128
IMAGE_HEIGHT = IMAGE_WIDTH
IMAGE_DEPTH = 3

# allow to build our image database
def build_image_database(path, target, encoding='UTF-8'):
    """Build a pandas dataframe with target class and access path to images.

    Parameters
    - - - - - -
    path (Path): path patern to read csv file containing images information.
    target (str): name of the target column.

    Returns
    - - - - -
    A pandas dataframe, including target class and path to image.
    """


    _df = pd.read_csv(path, sep=';', dtype={'all': str}, encoding=encoding)


    _df['path'] = _df['id'].apply(lambda x:  pathlib.Path(r'data_facture/all_facture') / (str(x) + '.png'))

    return _df

In [ ]:
def build_x_svm_and_y_svm(df:pd.DataFrame,target:str,images:str,stratify=None):
  """Build x tensor and y tensor for model fitting

  Parameters
  --------------
  df(pd.DataFrame):dataframe contanining images and target
  target(str):name of target column
  images(str): name of images column

  Returns
  ---------
  x(np.array):tensor of x values
  y(np.array):name of y values
  """
  x = pd.DataFrame(np.array([img for img in df[images]]))
  y = df[target].astype('category').cat.codes


  return x, y

In [ ]:
def load_resize(path,height,width):
  """Load an image and resize it to the target size
    Parameters
    --------
    path(Path): access path to image file
    height(int): resize image to this height
    width(int):resize image to this width

    Returns
    ----------
    nb.array containing resize


  """
  #np.array(Image.open(path).convert('RGB').resize((width, height))), (resize(imread(path)/255,(width,height))).flatten()
  return (resize(imread(path)/255,(width,height))).flatten()

In [ ]:
# Load train & test dataset

train_df= build_image_database('data_facture/data_train.csv','label')
test_df= build_image_database('data_facture/data_test1.csv','label')

train_df['resized_image']=train_df.apply(lambda r:load_resize(r['path'],IMAGE_HEIGHT,IMAGE_WIDTH),axis=1)

test_df['resized_image']=test_df.apply(lambda r:load_resize(r['path'],IMAGE_HEIGHT,IMAGE_WIDTH),axis=1)

X_train,y_train  =build_x_svm_and_y_svm(train_df,'label','resized_image')
X_test,y_test =build_x_svm_and_y_svm(test_df,'label','resized_image')

In [ ]:
X_train.describe()

In [ ]:
train_df

In [ ]:
test_df.head()

In [ ]:
# Defining the parameters grid for GridSearchCV
param_grid={'C':[0.1, 1, 10, 100, 1000, 10000, 100000],
            'kernel':['rbf', 'sigmoid', 'linear', 'poly']}

# Creating a support vector classifier
svc=svm.SVC(probability=True)

# Creating a model using GridSearchCV with the parameters grid
model=GridSearchCV(svc,param_grid)

In [ ]:
l=[]
for i in range(len(train_df)):
  l.append([x for x in train_df.loc[i, "resized_image"]])

In [ ]:
d= pd.DataFrame(l)

In [ ]:
d

In [ ]:
d=d.dropna(axis=1)

In [ ]:
pca = PCA(n_components=11)

In [ ]:
pca.fit(d)

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
X_train_svm = pca.transform(d)

In [ ]:
model.fit(X_train_svm,y_train)

In [ ]:
model.best_params_

In [ ]:
m=[]
for i in range(len(test_df)):
  m.append([x for x in test_df.loc[i, "resized_image"]])

In [ ]:
m= pd.DataFrame(m)

In [ ]:
m=m.dropna(axis=1)

In [ ]:
m

In [ ]:
X_test = pca.transform(m)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_pred, y_test)

In [ ]:
accuracy

In [ ]:
classification = classification_report(y_test, y_pred)

In [ ]:
print(classification)

In [ ]:
metrics.confusion_matrix(y_pred, y_test)

In [ ]:
import datetime
def save_model(model, basename):
  """Save tf/Keras model

  Model file is named model + timestamp.

  Parameters
  ----------
  model (tf/Keras model): model to be saved
  basename: location to save model file
  """
  model('{}.sav'.format(basename))
  return

In [ ]:
filename = 'model_vgg/model.sav'

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open(filename,"wb"))

In [ ]:
with open(filename, 'rb') as handle:
        model = pickle.load(handle)
        



In [ ]:
X_test[135]

In [ ]:
model.predict(X_test[40].reshape(-1,11))

In [ ]:
#création du excel pour visualiser sur qu'elle facture il c'est tromper

In [ ]:
test_df.to_excel('res_excel/test.xlsx')

In [ ]:
df_ypred = pd.DataFrame(y_pred)

In [ ]:
df_ypred.to_excel('res_excel/test_pred.xlsx')

In [ ]:
y_test.to_excel('res_excel/test_image.xlsx')